In [38]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
import scipy
import pandas as pd
import numpy as np
import os

In [40]:
# 读取Excel文件
df1 = pd.read_excel('/home/maxdmx/math-China/附件1(1).xlsx')
column_name_1 = '地块名称'
column_data_1 = df1[column_name_1].tolist()

df2 = pd.read_excel('/home/maxdmx/math-China/附件1(2).xlsx')
column_name_2 = '作物名称'
column_data_2 =df2[column_name_2].tolist()
column_data_1

['A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'B12',
 'B13',
 'B14',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'E1',
 'E2',
 'E3',
 'E4',
 'E5',
 'E6',
 'E7',
 'E8',
 'E9',
 'E10',
 'E11',
 'E12',
 'E13',
 'E14',
 'E15',
 'E16',
 'F1',
 'F2',
 'F3',
 'F4']

In [41]:
years = [1,2,3,4,5,6,7]  # 2024到2030年
seasons = [1, 2]  # 两个种植季节
crops = list(range(1, 42))  # 41种作物
fields = list(range(1, 55))  # 54块地

In [42]:
df3 = pd.read_excel('/home/maxdmx/math-China/附件2(2)1.xlsx')

npyields = np.zeros((42,55,3))
npcosts = np.zeros((42,55,3))
# 遍历每一行并提取所需的列
for index, row in df3.iterrows():
    crop_id = row['作物编号']        # 提取作物编号
    plot_type = row['地块类型']      # 提取地块类型
    planting_season = row['种植季次']  # 提取种植季次
    yield_per_acre = row['亩产量/斤']  # 提取亩产量
    cost_per_acre = row['种植成本/(元/亩)']
    if plot_type == '平旱地':
        listtemp1=[1,2,3,4,5,6]
    elif plot_type == '梯田':
        listtemp1=[7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    elif plot_type == '山坡地':
        listtemp1=[21,22,23,24,25,26]
    elif plot_type == '水浇地':
        listtemp1=[27,28,29,30,31,32,33,34]
    elif plot_type == '普通大棚':
        listtemp1=[35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
    elif plot_type == '智慧大棚':
        listtemp1=[51,52,53,54]

    if planting_season=='单季' or planting_season=='第一季':
        ps=1
    else:
        ps=2
    for t in listtemp1:
        npyields[crop_id,t,ps]=yield_per_acre
        npcosts[crop_id,t,ps]=cost_per_acre
costs=npcosts.tolist()
yields=npyields.tolist()

In [43]:
df4 = pd.read_excel('/home/maxdmx/math-China/Ec.xlsx')

column_name_3 = '作物编号'
column_name_4 = '2023总产量'
column_data_3 = df4[column_name_3].tolist()
column_data_4 = df4[column_name_4].tolist()
npexpected_sales=np.zeros(42)
for i,j in zip(column_data_3,column_data_4):
    npexpected_sales[i]=j
expected_sales=npexpected_sales.tolist()

In [34]:
df5 = pd.read_excel('/home/maxdmx/math-China/updated_附件2(2)1.xlsx')
column_name_5 = '作物编号'
column_name_6 = '销售单价平均值/(元/斤)'
column_data_5 = df5[column_name_5].tolist()
column_data_5 = column_data_5[:-18]
column_data_6 = df5[column_name_6].tolist()
column_data_6 = column_data_6[:-18]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41]

In [44]:
npprices=np.zeros(42)
for i,j in zip(column_data_5,column_data_6):
    npprices[i]=j
prices=npprices.tolist()

In [74]:
def objective(X_flat):
    X = X_flat.reshape(42, 55, 3, 8)  # 重新调整 X 的形状为 4 维数组
    profit = 0
    for c in crops:
        for y in years:
            sales_fake=0
            for s in seasons:
                for l in fields:
                    # 计算每个作物的销售量和利润
                    sales_fake += npyields[c, l, s] * X[c, l, s, y]
            sales=min(sales_fake,npexpected_sales[c])
            Cost=0
            for s in seasons: 
                for l in fields:
                    Cost+=npcosts[c,l,s]*X[c, l, s, y]
            profit += sales * npprices[c] - Cost
    return -profit  # 因为 minimize 是最小化，因此我们最大化 profit 时返回其负值

In [75]:
data = pd.read_excel('/home/maxdmx/math-China/附件1(1).xlsx')
field_areas = data['地块面积/亩'].to_numpy()
field_areas = np.insert(field_areas, 0, 0)
field_areas

array([ 0. , 80. , 55. , 35. , 72. , 68. , 55. , 60. , 46. , 40. , 28. ,
       25. , 86. , 55. , 44. , 50. , 25. , 60. , 45. , 35. , 20. , 15. ,
       13. , 15. , 18. , 27. , 20. , 15. , 10. , 14. ,  6. , 10. , 12. ,
       22. , 20. ,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,
        0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6])

In [76]:
def constraint_field_area(X_flat):
    # 将一维的 X_flat 重塑为 4 维数组 (crops, fields, seasons, years)
    X = X_flat.reshape(42, 55, 3, 8)
    
    constraints = []
    for l in range(1,55):  # 遍历所有地块
        for y in range(1,8):  # 遍历所有年份
            for s in range(1,3):  # 遍历所有季节
                total_area = np.sum([X[c, l, s, y] for c in range(1,42)])  # 计算该地块所有作物的种植总面积
                constraints.append(field_areas[l] - total_area)  # 添加约束条件，要求总面积 <= 地块面积
    return np.array(constraints)

In [77]:
def constraint_bean_crops(X_flat):
    X = X_flat.reshape(42, 55, 3, 8)
    constraints = []
    beans = [1, 2, 3, 4, 5, 17, 18, 19] 
    for l in fields:
        for s in seasons:
            for y in years[:-2]:
                total_bean_area = np.sum([X[bean, l, s, y + i] for bean in beans for i in range(3)])
                constraints.append(total_bean_area - 0.01)
    return np.array(constraints)

In [78]:
def constraint_crop_distribution(X_flat):
    X = X_flat.reshape(42, 55, 3, 8)  # 将 X_flat 重塑为 4 维数组
    max_field_count=6
    constraints = []
    for y in range(1,8):  # 遍历所有年份
        for s in range(1,3):  # 遍历所有季节
            for c in range(1,42):  # 遍历所有作物
                # 计算该作物在当前年份和季节种植的地块数量
                field_count = np.sum([X[c, l, s, y] > 0 for l in range(1,55)])
                # 添加约束条件，地块数量 <= max_field_count
                constraints.append(max_field_count - field_count)  # 确保地块数量不超过限制
    return np.array(constraints)

In [79]:
def constraint_area_minimize(X_flat):
    X = X_flat.reshape(42, 55, 3, 8)  # 将 X_flat 重塑为 4 维数组
    min_field_area=0.3
    constraints = []
    for y in range(1,8):  # 遍历所有年份
        for s in range(1,3):  # 遍历所有季节
            for c in range(1,42):  # 遍历所有作物
                for l in range(1,55):
                    if X[c, l, s, y] !=0:
                        constraints.append(X[c,l,s,y] - min_field_area ) 
    return np.array(constraints)

In [80]:

def constraint_rotation_flatland_eq(X_flat):
    X = X_flat.reshape(42, 55, 3, 8)  # 重新调整 X 为 4 维数组
    constraints = []
    for c in range(1, 16):  # 遍历作物
        for l in range(1, 27):  # 遍历平旱地
            for y in range(1,7):  # 遍历年份（7表示2023到2029）
                # 确保连续两年不能种植同一作物
                constraints.append(X[c, l, 1, y] * X[c, l, 1, y + 1])
    return np.array(constraints)

In [81]:
def constraint_rotation_greenhouse_eq(X_flat):
    X = X_flat.reshape(42, 55, 3, 8)  # 重新调整 X 为 4 维数组
    
    constraints = []
    for c in range(1, 16):  # 遍历作物
        for l in range(51, 55):  # 遍历智慧大棚
            for y in range(1,7):  # 遍历年份（7表示2023到2029）
                # 确保连续两年在同一地块的同一季节不能种植同一作物
                constraints.append(X[c, l, 1, y] * X[c, l, 1, y + 1])
                constraints.append(X[c, l, 2, y] * X[c, l, 2, y + 1]) 
    return np.array(constraints)

In [82]:
def constraint_flatland_terrace_hill(X_flat):
    X = X_flat.reshape(42, 55, 3, 8)  # 重新调整 X 为 4 维数组

    constraints = []
    for c in range(1, 16):  # 作物编号 1 到 15
        for l in range(1, 27):  # 平旱地、梯田、山坡
            for y in range(1,8):  # 遍历所有年份
                constraints.append(X[c, l, 2, y])  # 限制季节 2 不能种植该作物

    for c in range(17, 42):  # 作物编号 17 到 41
        for l in range(1, 27):  # 平旱地、梯田、山坡
            for s in range(1, 3):  # 遍历季节 1 和 2
                for y in range(1,8):  # 遍历所有年份
                    constraints.append(X[c, l, s, y])  # 该作物不能在这些季节种植

    return np.array(constraints)

In [83]:
def constraint_irrigated_land(X_flat):
    X = X_flat.reshape(42, 55, 3, 8)  # 重新调整 X 为 4 维数组

    constraints = []
    for l in range(27, 35):  # 水浇地
        for y in range(1,8):  # 遍历所有年份
            for c in range(1, 16):  # 作物编号 1 到 16
                constraints.append(X[c, l, 1, y])  # 限制季节 1 不能种植
                constraints.append(X[c, l, 2, y])  # 限制季节 2 不能种植

    return np.array(constraints)

In [84]:
def constraint_normal_greenhouse(X_flat):
    X = X_flat.reshape(42, 55, 3, 8)  # 重新调整 X 为 4 维数组

    constraints = []
    for l in range(35, 51):  # 普通大棚类
        for y in range(1,8):  # 遍历所有年份
            for c in range(1, 16):  # 作物编号 1 到 16
                constraints.append(X[c, l, 1, y] )  # 限制季节 1 不能种植
            for c in range(37, 42):  # 作物编号 37 到 41
                constraints.append(X[c, l, 2, y] )  # 限制季节 2 不能种植

    return np.array(constraints)

In [85]:
def constraint_smart_greenhouse(X_flat):
    X = X_flat.reshape(42, 55, 3, 8)  # 重新调整 X 为 4 维数组

    constraints = []
    for l in range(51, 55):  # 智慧大棚类
        for s in range(1, 3):  # 遍历季节 1 和 2
            for y in range(1,8):  # 遍历所有年份
                for c in range(1, 16):  # 作物编号 1 到 16
                    constraints.append(X[c, l, s, y] == 0)  # 不能种植该作物

    return np.array(constraints)

In [88]:
# 定义约束条件的形式：type 为 'ineq' 表示这是一个不等式约束
constraints = [
    {'type': 'ineq', 'fun': constraint_field_area},
    # {'type': 'ineq', 'fun': constraint_bean_crops},
    # {'type': 'ineq', 'fun': constraint_crop_distribution},
    # {'type': 'ineq', 'fun': constraint_area_minimize},
    # {'type': 'eq', 'fun': constraint_rotation_flatland_eq},
    # {'type': 'eq', 'fun': constraint_rotation_greenhouse_eq},
    # {'type': 'eq', 'fun': constraint_flatland_terrace_hill},
    # {'type': 'eq', 'fun': constraint_irrigated_land},
    # {'type': 'eq', 'fun': constraint_normal_greenhouse},
    # {'type': 'eq', 'fun': constraint_smart_greenhouse},
]

In [89]:
from scipy.optimize import minimize

# 初始变量X0，可以根据你的问题规模初始化
X0 = np.zeros(42 * 55 * 3 * 8)  # 根据具体需求，初始化为一维数组

# 使用SciPy的minimize函数进行优化
result = minimize(objective, X0, method='trust-constr', constraints=constraints, options={'verbose': 1})

# 输出优化结果
print("优化完成：", result)

KeyboardInterrupt: 